Using a LSTM single model to text various cleaning steps and impact on score.

Controls:
- CNN single model
- maxlen: 65
- min occurance vocab: 5
- glove.6B.100D
- epochs: 2
- cv: 3
- max features 20000

In [1]:
model_name = 'grid_'

In [2]:
import os

In [3]:
dir_path = os.path.realpath('..')

In [4]:
# Import custom transformers

path = 'src/features'
full_path = os.path.join(dir_path, path)
import sys
sys.path.append(full_path)
from transformers import TextCleaner, KerasProcesser

Using TensorFlow backend.


## Import data

In [5]:
import numpy as np
import pandas as pd

In [6]:
path = 'data/raw/train.csv'

full_path = os.path.join(dir_path, path)
df_train = pd.read_csv(full_path, header=0, index_col=0)
print("Dataset has {} rows, {} columns.".format(*df_train.shape))

Dataset has 95851 rows, 7 columns.


In [7]:
# fill NaN with string "unknown"
df_train.fillna('unknown',inplace=True)

## Pre-processing

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
seed = 42
np.random.seed(seed)
test_size = 0.2
target = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
corpus = 'comment_text'

X = df_train[corpus]
y = df_train[target]


Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=test_size, random_state=seed)

In [10]:
max_features=20000
max_length=65

## Model fit

In [11]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline

In [12]:
from keras.models import Sequential
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation

# Function to create model, required for KerasClassifier
def create_model(optimizer='adam', max_features=max_features, max_length=max_length):
    model = Sequential()
    model.add(Embedding(max_features, 100, input_length=max_length))
    model.add(Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)))
    model.add(GlobalMaxPool1D())
    model.add(Dense(50, activation="relu"))
    model.add(Dropout(0.1))
    model.add(Dense(6, activation='sigmoid'))  #multi-label (k-hot encoding)
    # compile network
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [13]:
def save_model(model, model_path):
    # serialize model to JSON
    model_json = model.to_json()
    with open(model_path + ".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(model_path + ".h5")
    print("Saved model to disk")

In [14]:
model = KerasClassifier(build_fn=create_model, epochs=3, verbose=2)

In [15]:
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.tokenize import RegexpTokenizer

class TextCleaner(BaseEstimator, TransformerMixin):
    def __init__(self, regex=r'[\w]+', remove_digits=False, english_only=False, stop_words=None, lower=True):
        self.regex = regex
        self.remove_digits = remove_digits
        self.english_only = english_only
        self.stop_words = stop_words
        self.lower = lower
        
    def transform(self, X, *args):
        tokenizer = RegexpTokenizer(self.regex)
        result = []
        for row in X:
            tokens = tokenizer.tokenize(row)
            if self.lower:
                tokens = [t.lower() for t in tokens]
            if self.remove_digits:
                tokens = [t for t in tokens if not t.isdigit()]
            if self.english_only:
                english_words = set(nltk.corpus.words.words())
                tokens = [t for t in tokens if t in english_words]
            if self.stop_words is not None:
                tokens = [t for t in tokens if not t in self.stop_words]
            tokens = ' '.join(tokens)
            if tokens == '':
            	tokens = 'cleaned'
            result.append(tokens)
        return result
    
    def fit(self, *args):
        return self

In [18]:
p = Pipeline([
    ('cleaner', TextCleaner()),
    ('keraser', KerasProcesser(num_words=max_features, maxlen=max_length)),
    ('clf', model)
])

param_grid = {"cleaner__regex": ['\S+'],
              "cleaner__remove_digits": [False],
              "cleaner__english_only": [False],
              "cleaner__stop_words": [None],
              "cleaner__filters": ['[!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n]'],
              "cleaner__lower": [True]
             }

In [ ]:
%%time
grid = GridSearchCV(p, param_grid=param_grid, cv=2)
grid_result = grid.fit(Xtrain, ytrain)

In [ ]:
trained_model = grid_result.best_estimator_.named_steps['clf'].model

In [ ]:
# save the model
model_path = os.path.join(dir_path, 'models', model_name)
save_model(trained_model, model_path)

## Evaluation

In [ ]:
# summarize results
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
print("Best score {} with params {}".format(grid_result.best_score_, grid_result.best_params_))